In [36]:
import re
import time
import numpy as np
import pandas as pd
import translators as ts
from googletrans import Translator, LANGUAGES

translator = Translator()

def fetch_column_names(row):
    ## FETCHING DATAFRAME COLUMNS
    columns = row[row.find('('):row.find(')')]
    # Characters to remove
    chars_to_remove = '()[] '
    # Create a translation table: each character to be removed maps to None
    trans_table = str.maketrans('', '', chars_to_remove)
    columns = columns.translate(trans_table).split(',')
    return columns

def fetch_values(row):
    values = row[row.find("VALUES")+8:-2]
    pattern = pattern = r", (?=(?:[^']*'[^']*')*[^']*$)"
    values = re.split(pattern, values)
    return values
    
def permeate_values_on_df(columns, raw_data):
    # Assuming 'data' is a list of strings
    df_raw = pd.DataFrame(raw_data, columns=['raw_data'])
    # Example parsing operation: extracting part after a delimiter
    df_raw['raw_data'] = df_raw['raw_data'].apply(fetch_values)
    df_new = pd.DataFrame(columns=columns)
    df_new[columns] = pd.DataFrame(df_raw['raw_data'].to_list())
    return df_new

# def translate(text):

# Function to translate text with delay to avoid rate limits
def translate_text_with_delay(text, delay=1):
    raw_unstranslated_text = re.findall(r'\'(.*?)\'', text)
    translated_text = ''
    if len(raw_unstranslated_text[0]):
        translated_text_en = ts.translate_text(query_text=raw_unstranslated_text[0], translator = 'bing', from_language = 'auto', to_language = 'en')
        translated_text = ts.translate_text(query_text=translated_text_en, translator = 'bing', from_language = 'en', to_language = 'pt')
        time.sleep(delay)  # Wait for a specified delay (in seconds) between requests
    original_format_translated_text = f"N'{translated_text}'"
    return original_format_translated_text

def translate_df_columns(df):
    columns_to_translate = ['Name', 'Description']

    for column in columns_to_translate:
        df[column] = df[column].apply(translate_text_with_delay)
    return df

In [24]:
with open('teste 500 linhas.sql', 'r', encoding='utf-8') as f:
    raw_data = f.readlines()
raw_data = [item for item in raw_data if item!='\n']
columns = fetch_column_names(raw_data[0])
df = permeate_values_on_df(columns, raw_data)
df.iloc[0:2]

,TemplateID,Name,Remark,CategoryID,Description,Attack,Defence,Agility,Luck,Level,...,FusionNeedRate,Hole,RefineryLevel,ReclaimValue,ReclaimType,CanRecycle,FloorPrice,SuitId,CanTransfer,Price
0,1392,N'Kanada geyiği şapkası',N'',1,N'',5,5,5,5,3,...,0,"N'3,3|6,2|9,3|12,3|0,3|0,3'",0,20,1,1,0,0,1,NULL
1,2506,N'Học Sĩ',N'',2,"N'Nằm trong bộ Học Viện Phép Thuật (Nam), có t...",8,8,8,8,3,...,0,"N'0,-1|0,-1|0,-1|0,-1|0,-1|0,-1'",-1,15,1,0,0,0,1,NULL


In [37]:
df_translated = translate_df_columns(df.iloc[0:1])
df_translated.iloc[0:1]

['Kanada geyiği şapkası']
['']


C:\Users\clib_\AppData\Local\Temp\ipykernel_22284\114467822.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(translate_text_with_delay)


,TemplateID,Name,Remark,CategoryID,Description,Attack,Defence,Agility,Luck,Level,...,FusionNeedRate,Hole,RefineryLevel,ReclaimValue,ReclaimType,CanRecycle,FloorPrice,SuitId,CanTransfer,Price
0,1392,N'Chapéu de alce canadense',N'',1,N'',5,5,5,5,3,...,0,"N'3,3|6,2|9,3|12,3|0,3|0,3'",0,20,1,1,0,0,1,NULL
